In [2]:
import numpy as np
import pyvista as pv
from irregular_object_packing.performance_analysis import plots
from irregular_object_packing.cat.tetrahedral_split import (
    split_2_2222,
    split_2_3331,
    split_3,
    split_4,

)


In [16]:
cylinder = pv.Cylinder(height=2)
plotter = pv.Plotter()
plotter.add_mesh(cylinder, color="white", opacity=0.5)
plotter.add_mesh(pv.Box(cylinder.bounds).extract_all_edges(), color="black")
plotter.isometric_view()
plotter.save_graphic("renders/cylinder.pdf")
plotter.show(interactive=False)


Widget(value="<iframe src='http://localhost:53368/index.html?ui=P_0x17fe3a350_12&reconnect=auto' style='width:…

In [4]:
from itertools import combinations

from irregular_object_packing.mesh.utils import convert_faces_to_polydata_input


tet1 = np.array([ # faces allign with axis
    [0, 0, 0],
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
])
tet1 = pv.Tetrahedron().points

tet2 = np.array([ # pointy end alligns with axis
    [0, 0, 0],
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 5],
])
tet3 = np.array([ # pointy flat equilateral
    [0, 0, 0],
    [1, 0, 0],
    [0.5, 0.866, 0],
    [0.5, 0.288, 0.1],
])
tet4 = np.array([ # 
    [0, 0, 0],
    [5, 0, 0],
    [0, 1, 0],
    [0, 0, 5],
])
def mesh_from_tet(tet:np.ndarray): 
    faces = list(combinations(tet, 3))
    return pv.PolyData(*convert_faces_to_polydata_input(faces))

    
plotter = pv.Plotter()
plotter.add_mesh(mesh_from_tet(tet1), color='white',  opacity=0.5)
plotter.add_mesh(mesh_from_tet(tet1).extract_all_edges(), color='black',show_edges=True, edge_color="black", line_width=5,  opacity=1)
plotter.view_isometric()
plotter.view_vector((-1,-1,2))
plotter.show()
plotter.save_graphic('renders/tet.pdf')

Widget(value="<iframe src='http://localhost:60863/index.html?ui=P_0x105994af0_1&reconnect=auto' style='width: …

In [3]:
from matplotlib.colors import hex2color
from irregular_object_packing.mesh.utils import convert_faces_to_polydata_input

colorslist = [
    ["red", "red", "blue", "blue"],
    ["red", "red", "red", "blue"],
    ["red", "red", "blue", "green"],
    ["red", "green", "blue", "yellow"]
]
viewup = (-1,-1,1)
splits = [ split_2_2222, split_2_3331, split_3, split_4 ]
tets = [ tet1,]#tet2, ]

# plotter = pv.Plotter(shape = (len(tets),len(splits)),)
for i, split in enumerate(splits):
    for j, tet in enumerate(tets):
        plotter =  pv.Plotter()
        # plotter.subplot(j,i)
        faces = split(tet)
        colors = colorslist[i]
        color_id = 0
        
        plotter.add_mesh(mesh_from_tet(tet).extract_all_edges(), color="black", show_edges=True, line_width=5,  opacity=1)
        plotter.add_mesh(mesh_from_tet(tet), color="white", opacity=0.1)
        for k, face in enumerate(faces):
            plotter.add_points(np.array([tet[k]]), color=colors[k], point_size=25, render_points_as_spheres=True, opacity=1)
        plotter.view_isometric()
        plotter.view_vector((1,1,2))
        plotter.save_graphic(f"renders/tet_{split.__name__}_points.pdf")
        
        for k, face in enumerate(faces):
            plotter.add_mesh(pv.PolyData(*convert_faces_to_polydata_input(face)), color='#D3D3D3', edge_color='purple', show_edges=True, show_vertices=True,point_size=10, render_points_as_spheres=True,line_width=15, opacity=1)
            plotter.add_points(pv.PolyData(*convert_faces_to_polydata_input(face)).points, color='purple', point_size=22, render_points_as_spheres=True, opacity=1)

        plotter.view_isometric()
        plotter.view_vector((-1,-1,2))
        plotter.save_graphic(f"renders/tet_{split.__name__}.pdf")

        # generate orbital path gif
        path = plotter.generate_orbital_path(n_points=60,viewup=viewup,)#shift=0.2)
        plotter.open_gif(f"renders/tet_{split.__name__}.gif")
        plotter.orbit_on_path(path, viewup=viewup, write_frames=True)
        plotter.close()

        
# plotter.show()